# Filmdatenbank mithilfe von SPARQL mit OpenData erweitern

### Disclaimer
Um den Code in diesem Notebook ausführen zu können, müssen vorher einige Packages installiert, importiert und eine Verbindung zur MongoDB aufgebaut werden. Dazu muss der nachstehende Code Block ausgeführt werden.

In [ ]:
%pip install pymongo
%pip install SPARQL-Burger
%pip install requests
from SPARQLBurger.SPARQLQueryBuilder import *
import requests
import pandas as pd
import pymongo
from pprint import pprint
client = pymongo.MongoClient("mongodb://mongodb:27017")
db = client['moviedb']
collection = db['movie']

## 1. Aufgabe

### 1.1 Beschreibung
Eine vorgegebenen Filmdatenbank für MongoDB soll mit OpenData erweitert werden. Als Datenbasis wird [Wikidata](https://wikidata.org/) genutzt, 
welche einer der größten offenen und kollaborativ erstellten Wissensdatenbanken der **Welt** ist. Die Filmdatenbank soll mit ausgewählten Informationen ergänzt werden und die neuen Daten dargestellt werden. Schlussendlich sollen neuen Abfragemöglichkeiten gezeigt werden, die sich aufgrund der neuen Daten ergeben.

### 1.2 Beispielhafter Datensatz
```json
{
  "_id": {
    "$oid": "646787ea309412c96f14b0c4"
  },
  "index": 0,
  "adult": "False",
  "belongs_to_collection": "{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",
  "budget": "30000000",
  "genres": "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",
  "homepage": "http://toystory.disney.com/toy-story",
  "id": "862",
  "imdb_id": "tt0114709",
  "original_language": "en",
  "original_title": "Toy Story",
  "overview": "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
  "popularity": 21.946943,
  "poster_path": "/rhIRbceoE9lR4veEXuwCC2wARtG.jpg",
  "production_companies": "[{'name': 'Pixar Animation Studios', 'id': 3}]",
  "production_countries": "[{'iso_3166_1': 'US', 'name': 'United States of America'}]",
  "release_date": "1995-10-30",
  "revenue": 373554033,
  "runtime": 81,
  "spoken_languages": "[{'iso_639_1': 'en', 'name': 'English'}]",
  "status": "Released",
  "tagline": {
    "$numberDouble": "NaN"
  },
  "title": "Toy Story",
  "video": false,
  "vote_average": 7.7,
  "vote_count": 5415,
}
```

### 1.3 Umsetzung

### 1.3.1 Beispielhafter Durchlauf zum Laden der Daten mit SPARQL

### 1.3.2 Datensatz mit erweiterten Informationen

## 2. Abfragen mit der Datenbank

### 2.1 Beispielhafte Abfragen

Alle Filme mit Schauspieler X oder Y

In [ ]:
movies = collection.find({'cast_member': { '$in': ['Kursten Dunst', 'Brad Pitt']}})
for movie in movies:
    pprint(movie['title'])
    pprint(movie['cast_member'])
    pprint('-----------------------')

Alle Filme mit Thema X und Y

In [ ]:
movies = collection.find({'main_subject': { '$all': ['computing', 'artificial intelligence']}})
for movie in movies:
    pprint(movie['title'])
    pprint(movie['main_subject'])
    pprint('-----------------------')

Alle Filme mit einem Vorgänger oder Nachfolger

In [ ]:
movies = collection.find({'$or': [ {'follows': {'$ne': ''}}, {'followed_by': {'$ne': ''}}]})
for movie in movies:
    pprint(f'Vorgänger: {movie.get("follows", "")}')
    pprint(f'Nachfolger: {movie.get("followed_by", "")}')
    pprint('-----------------------')

Alle Filme mit Kosten zwischen X und Y

In [ ]:
movies = collection.find({'$and': [
     {'cost': {'$ne': ''}},
     {'$expr': {'$gte': [{'$toDouble': '$cost'}, 10000000]}},
     {'$expr': {'$lte': [{'$toDouble': '$cost'}, 15000000]}} 
     ]})
for movie in movies:
    pprint(movie['cost'])

### 2.2 Eure Abfragen

In [ ]:
movies = collection.find({})
for movie in movies:
    pprint(movie)